<a href="https://colab.research.google.com/github/markNZed/GPT-NeoX-Colab/blob/venv/notebooks/shakespeare_trainingV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from datetime import datetime
print("Current Date and Time:", datetime.now())

Current Date and Time: 2024-11-08 13:41:39.051959


In [3]:
%%time
%cd /content
# Don't use --depth 1 because that does not play nice with git-annex
!git clone https://github.com/markNZed/GPT-NeoX-Colab.git
%cd /content/GPT-NeoX-Colab
!pip install . >& /dev/null
from dotenv import load_dotenv
import os
load_dotenv('/content/GPT-NeoX-Colab/.env')
from GPTNeoXColab import utils
utils.install_git_annex()
utils.enable_remote()
utils.sync_and_get_data()

/content
fatal: destination path 'GPT-NeoX-Colab' already exists and is not an empty directory.
/content/GPT-NeoX-Colab
git-annex is already installed.
Starting git annex sync...
Sync successful. Fetching data from backblaze...
Data retrieval successful.
CPU times: user 211 ms, sys: 42.5 ms, total: 253 ms
Wall time: 29.5 s


In [ ]:
#@title Mount Google Drive
SAVE_TO_DRIVE = False
CACHE_PIP = False
if SAVE_TO_DRIVE or CACHE_PIP:
  from google.colab import drive
  drive.mount('/content/drive')
  !mkdir -p /content/drive/MyDrive/pip_cache
  !mkdir -p /content/drive/MyDrive/pip_wheels
  %env PIP_CACHE_DIR=/content/drive/MyDrive/pip_cache
  %env PIP_FIND_LINKS=/content/drive/MyDrive/pip_wheels

In [ ]:
#@title SSH Connection for VS Code
SSH_CONNECT = False
if SSH_CONNECT:
  from google.colab import userdata
  if "SSH_CONNECTED" not in globals():
      SSH_CONNECTED = None
  # This is using the Colab "Secrets" feature
  if userdata.get('REMOTE_SSH') and not SSH_CONNECTED:
    SSH_CONNECTED = True
    !apt-get install -y openssh-server
    !mkdir -p /var/run/sshd
    !echo 'root:root' | chpasswd  # Set the root password to 'root'
    !echo 'PermitRootLogin yes' >> /etc/ssh/sshd_config
    !service ssh restart
    !apt-get install -y screen
    import pexpect
    import getpass
    password = getpass.getpass("Enter your SSH password: ")
    ssh_command = f"ssh -N -R 2223:localhost:22 -o StrictHostKeyChecking=no -o ServerAliveInterval=60 -o ServerAliveCountMax=5 {userdata.get('REMOTE_SSH')}"
    child = pexpect.spawn(ssh_command, encoding='utf-8')
    child.expect(["password:", "Password:"], timeout=60)
    child.sendline(password)
    child.sendline("") # Send an empty command to prevent immediate exit
    print("SSH session is running in the background.")
    # AFter this the client should also connect to the notebook SSH via the forwarded port
    # ssh -N -L 9999:localhost:2223 -o ServerAliveInterval=60 -o ServerAliveCountMax=5 REMOTE_SSH


# Cloning GPT-NeoX Repo

In [ ]:
%%time
%cd /content
# Fix for text_generation
#!git clone https://github.com/EleutherAI/gpt-neox
!git clone -b text_gen_not_parallel https://github.com/markNZed/gpt-neox.git

In [ ]:
%%time
# Experimenting with BackBlaze
BACKBLAZE_SAVE = False
%cd /content
!pip install boto3
# Install pigz and pv for multi-threaded compression and progress tracking
!apt-get install -y pigz pv
import boto3
from botocore.exceptions import ClientError
from botocore.config import Config
import os
import sys
import traceback

# Upload specified file into the specified bucket
def upload_file(bucket, directory, file, b2, b2path=None):
    file_path = directory + '/' + file
    remote_path = b2path
    if remote_path is None:
        remote_path = file
    try:
        response = b2.Bucket(bucket).upload_file(file_path, remote_path)
    except ClientError as ce:
        print('error', ce)
        traceback.print_exc()  # Print the full stack trace
    return response

# Download the specified object from B2 and write to local file system
def download_file(bucket, directory, file, key_name, b2):
    file_path = directory + '/' + file
    try:
        b2.Bucket(bucket).download_file(key_name, file_path)
    except ClientError as ce:
        print('error', ce)
        traceback.print_exc()  # Print the full stack trace

# Return a boto3 resource object for B2 service
def get_b2_resource(endpoint, key_id, application_key):
    b2 = boto3.resource(
        service_name='s3',
        endpoint_url=endpoint,                # Backblaze endpoint
        aws_access_key_id=key_id,              # Backblaze keyID
        aws_secret_access_key=application_key, # Backblaze applicationKey
        config = Config(
            signature_version='s3v4',
        )
    )
    return b2

key_id_r = "003cb130fbeaa800000000001"
app_id_r = "K00347hujVVLL/TFqml/lXyhtkxB/C0"
b2_endpoint = "https://s3.eu-central-003.backblazeb2.com"
b2_r = get_b2_resource(b2_endpoint, key_id_r, app_id_r)
if BACKBLAZE_SAVE:
  from google.colab import userdata
  key_id_rw = "003cb130fbeaa800000000002"
  b2_rw = get_b2_resource(b2_endpoint, key_id_rw, userdata.get('B2_APP_KEY_RW'))
bucket = "GPT-NeoX-Colab"
directory = "/content"
if not os.path.exists("/content/my_env.tar") and not BACKBLAZE_SAVE:
  # Downloading takes about 40sec
  print("Downloading my_env.tar.gz")
  download_file(bucket, directory, "my_env.tar.gz", "my_env.tar.gz", b2_r)
  print("Unzipping my_env.tar.gz")
  !pigz -d -p 4 /content/my_env.tar.gz  # Decompress using 4 cores (adjust as needed)
  print("Untarring my_env.tar.gz")
  !pv /content/my_env.tar | tar -xf - -C /content

# Installing Dependencies

In [ ]:
%%bash
# Measure time with `time` command
time {
  # Check if the directory does not exist
  if [ ! -d "/content/my_env" ]; then
    # Install venv package for Python 3.10
    apt-get update && apt-get install -y python3.10-venv

    # Install virtualenv
    pip install virtualenv

    # Create and activate the virtual environment
    python3 -m venv /content/my_env
    source /content/my_env/bin/activate

    # Change to the GPT-NeoX directory
    cd /content/gpt-neox

    # Install specific versions of torch and other packages to avoid compatibility issues
    pip install torch==2.3.0 torchaudio==2.3.0 torchvision==0.18.0 transformers==4.41.0 sentence-transformers==2.2.2

    # Install dependencies with parallel jobs to speed up
    pip install -r ./requirements/requirements.txt
    pip install -r ./requirements/requirements-tensorboard.txt
  fi
}


In [ ]:
# tar and gzip the venv, takes about 2.5mins
# Check if the compressed file already exists
if not os.path.exists("/content/my_env.tar") and BACKBLAZE_SAVE:
    # Only run if my_env.tar.gz does not exist
    print("my_env.tar not found, proceeding with compression.")
    # Change directory (if needed) to ensure paths are correct
    %cd /content
    # Compress the tar archive with pigz, showing a progress indicator
    # Compresses 5.5G to 3.1G
    !time tar cf - my_env | pv -p -e -r -b | pigz -p 4 -1 > my_env.tar.gz
    # Upload to BackBlaze
    if userdata.get('B2_APP_KEY_RW'):
      response = upload_file(bucket, directory, "my_env.tar.gz", b2_rw)
      print("Upload response:", response)
    else:
      print("Skipping upload to BackBlaze.")
else:
    print("Skipping upload to BackBlaze.")

# Preparing Custom Dataset

In [ ]:
%cd /content/gpt-neox
!mkdir -p data

import json

# Generate a list of dictionaries
lines = []
with open("/content/GPT-NeoX-Colab/data/shakespeare.txt", encoding="utf8") as f:
    for line in f.read().splitlines():
        if line:
            lines.append({"text": line})

# Convert to a list of JSON strings
json_lines = [json.dumps(l) for l in lines]

# Join lines and save to .jsonl file
json_data = '\n'.join(json_lines)
with open('/content/gpt-neox/data/tinyshakespeare.jsonl', 'w') as f:
    f.write(json_data)

# TinyShakespeare Dataset

# Tokenizing Dataset

In [ ]:
%%time
%%bash
source /content/my_env/bin/activate
cd /content/gpt-neox
mkdir -p processed_data
python tools/datasets/preprocess_data.py \
    --input ./data/tinyshakespeare.jsonl \
    --output-prefix ./processed_data/tinyshakespeare \
    --tokenizer-type CharLevelTokenizer \
    --dataset-impl mmap \
    --append-eod

# Training

In [ ]:
%load_ext tensorboard

In [ ]:
import subprocess

# Start a detached background process
process = subprocess.Popen(
    "nohup bash -c \"source /content/my_env/bin/activate && python ./deepy.py train.py --conf_dir /content/GPT-NeoX-Colab/configs shakespeare.yml shakespeare_deepy.yml\" > /dev/null 2>&1 & echo $! > train_process.pid",
    shell=True,
    executable='/bin/bash',
    preexec_fn=subprocess.os.setsid  # Starts the process in a new session
)


In [ ]:
# Wait until a tensorboard is created
import time
import os

# Path to the TensorBoard log directory
tensorboard_log_dir = '/content/gpt-neox/tensorboard'

# Wait for the directory to be created
while not os.path.exists(tensorboard_log_dir):
    print("Waiting for TensorBoard log directory to be created...")
    time.sleep(10)  # Check every X seconds

print("TensorBoard log directory found. You can now launch TensorBoard.")


In [ ]:
# Need to remove everything in checkpoints and tensorboard dir for a fresh run
%tensorboard --logdir tensorboard

In [ ]:
import glob
import os

# Define the log directory and pattern for log files
log_dir = "/content/gpt-neox/logs"
log_pattern = os.path.join(log_dir, "*_stdout.txt")

# Get the list of log files that match the pattern
log_files = glob.glob(log_pattern)

# Ensure there are log files in the directory
if log_files:
    # Find the latest log file based on modification time
    latest_log = max(log_files, key=os.path.getmtime)
    print("Latest log file:", latest_log)
else:
    latest_log = None
    print("No log files found.")

# Now `latest_log` holds the path to the latest log file or None if not found


In [ ]:
import time
import os
import re

# File to store the last read position (persistence between script runs)
file_position = 0
# Regular expression to match "iteration <number> / <total>"
iteration_pattern = re.compile(r"iteration\s+(\d+)\s*/\s*\d+")

def read_new_iterations():
    global file_position
    # Open the log file and seek to the last position
    with open(latest_log, "r") as file:
        file.seek(file_position)
        # Read new lines
        new_lines = file.readlines()
        file_position = file.tell()
        # Process lines containing "iteration"
        last_match = None
        for line in new_lines:
            match = iteration_pattern.search(line)
            if match:
                last_match = match
        if last_match:
            # Extract the iteration count from the regex match
            iteration_count = int(last_match.group(1))
            print(f"{iteration_count} iterations")

# Read the PID from the file
with open("train_process.pid", "r") as f:
    pid = int(f.read().strip())
    print("PID:", pid)

# Function to check if the process is running
def is_process_running(pid):
    try:
        os.kill(pid, 0)  # Sending signal 0 to check if the process exists
        return True
    except OSError:
        return False

# Monitor the process
while is_process_running(pid):
    read_new_iterations()
    print("Training is still running...")
    time.sleep(30)  # Check every X seconds

print("Training has finished.")


In [ ]:
# Wait until a checkpoint is created
import time
import os

# Path to the checkpoints directory
checkpoints_dir = '/content/gpt-neox/checkpoints'

# Wait for the directory to be created
while not os.path.exists(checkpoints_dir):
    print("Waiting for checkpoints directory to be created...")
    time.sleep(10)  # Check every X seconds

print("Checkpoints directory found.")


In [ ]:
#@title Training and Validation Loss
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorboard.backend.event_processing import event_accumulator
import os
import numpy as np
# Path to the latest log file
log_dir = "tensorboard"
log_files = [os.path.join(log_dir, d) for d in os.listdir(log_dir)]
latest_log_dir = max(log_files, key=os.path.getmtime)

# Initialize EventAccumulator to load scalar data
ea = event_accumulator.EventAccumulator(latest_log_dir)
ea.Reload()  # Load all logs

# List all scalar keys available in the logs
scalar_keys = ea.Tags()['scalars']
print("Available scalar keys:", scalar_keys)

# Extract training and validation losses
train_loss = ea.Scalars('train/lm_loss')  # Adjust for actual name if necessary
val_loss = ea.Scalars('validation/lm_loss')  # Adjust for actual name if necessary

# Convert to lists for plotting
train_loss_values = [x.value for x in train_loss]
val_loss_values = [x.value for x in val_loss]

# Find the lengths of both arrays
len_train = len(train_loss_values)
len_val = len(val_loss_values)

iterations = None
# Interpolate the shorter array
if len_train != len_val:
    if len_train > len_val:
        # Interpolate validation loss to match the training loss length
        iterations = np.linspace(1, len_train, len_train)
        val_iterations = np.linspace(1, len_train, len_val)
        val_loss_values = np.interp(iterations, val_iterations, val_loss_values)
    else:
        # Interpolate training loss to match the validation loss length
        iterations = np.linspace(1, len_val, len_val)
        train_iterations = np.linspace(1, len_val, len_train)
        train_loss_values = np.interp(iterations, train_iterations, train_loss_values)
else:
    iterations = range(1, len_train + 1)

# Plot training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(iterations, train_loss_values, label='Training Loss')
plt.plot(iterations, val_loss_values, label='Validation Loss')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# If we want to monitor the logs
#!tail -f /content/gpt-neox/logs/*stdout.txt

# Inference with GPT-NeoX

In [ ]:
%%time
# This has issues if used during training -  The server socket has failed to bind to [::]:29500 (errno: 98 - Address already
# This will write over the logs
!source /content/my_env/bin/activate && python ./deepy.py generate.py -d configs /content/GPT-NeoX-Colab/configs/shakespeare.yml /content/GPT-NeoX-Colab/configs/shakespeare_gen.yml > /dev/null
!cat sample_output.txt

# Inference with Hugging Face

## Convert Our Model to HuggingFace Format
Here we are converting our model to `HuggingFace Format`. Follow the below instruction:
1. Add the `tokenizer_type` parameter to your .yml file:
(Example `"tokenizer_type": "CharLevelTokenizer",`)
In my use case I ll add above parameter to my `shakespeare.yml` file.
2. Select your best model from `checkpoints` directory. In my use case I select `global_step400`

In [ ]:
import os

# Define the path to the checkpoints directory
checkpoints_dir = "/content/gpt-neox/checkpoints"

# Read the 'latest' file to get the latest checkpoint name
with open(os.path.join(checkpoints_dir, "latest"), "r") as f:
    latest_checkpoint_name = f.read().strip()

# Construct the full path to the latest checkpoint directory
latest_checkpoint_path = os.path.join(checkpoints_dir, latest_checkpoint_name)
print("Path to the latest checkpoint:", latest_checkpoint_path)


In [ ]:
!source /content/my_env/bin/activate && python ./tools/ckpts/convert_neox_to_hf.py --input_dir {latest_checkpoint_path} --config_file /content/GPT-NeoX-Colab/configs/shakespeare.yml --output_dir hf_model/save/location --precision auto --architecture neox

## Generating Text

In [ ]:
from transformers import GPTNeoXForCausalLM
import torch

# Move to model directory
%cd /content/gpt-neox

# Assuming CharLevelTokenizer is properly imported and instantiated
from GPTNeoXColab import CharLevelTokenizer
tokenizer = CharLevelTokenizer()

# Load your model
model_path = "/content/gpt-neox/hf_model/save/location"
model = GPTNeoXForCausalLM.from_pretrained(model_path)

# Define a simple char-level tokenizer if not provided
def char_level_tokenize(text):
    return tokenizer.tokenize(text)

def char_level_detokenize(tokens):
    return tokenizer.detokenize(tokens)

# Set the model to evaluation mode
model.eval()

# Prompt the user for input
#input_text = input("Enter your prompt: ")
input_text = "Thou art"

# Tokenize and prepare input
input_ids = torch.tensor([char_level_tokenize(input_text)], dtype=torch.long)
attention_mask = torch.ones_like(input_ids)  # Create an attention mask for non-padded input

# Generate text with specified pad_token_id and attention_mask
with torch.no_grad():
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=200,          # Adjust this for desired output length
        temperature=0.7,        # Controls creativity
        top_k=50,               # Controls diversity
        top_p=0.9,              # Nucleus sampling
        num_return_sequences=1, # Number of sequences to return
        pad_token_id=model.config.eos_token_id,  # Set pad_token_id explicitly
        do_sample=True           # Enable sampling mode to use temperature and top_p
    )

# Decode and print the generated text
generated_text = char_level_detokenize(output[0].tolist())
print("Generated text:", generated_text)

In [ ]:
#@title Storing Checkpoints in Google Drive
if SAVE_TO_DRIVE:

  import shutil
  import os

  # Define source and destination base paths
  source_folder = '/content/gpt-neox/checkpoints'
  destination_folder = '/content/drive/MyDrive/gpt-neox-checkpoints'

  # Function to add version suffix if destination folder exists
  def get_versioned_folder_path(base_path):
      version = 1
      new_path = base_path
      while os.path.exists(new_path):
          new_path = f"{base_path}_v{version}"
          version += 1
      return new_path

  # Get the versioned destination folder path
  destination_folder_versioned = get_versioned_folder_path(destination_folder)

  # Copy the folder
  shutil.copytree(source_folder, destination_folder_versioned)

  print(f"Folder copied successfully to Google Drive as '{destination_folder_versioned}'!")